# Task
These files are for the human PPI networks. How to upload these files and build a graph through networkx

Here is all the data you need:
"NegativeEdges.txt"
"PositiveEdges.txt"
"proteinList.txt"
"sequenceList.txt"

## Data loading

### Subtask:
Load the four data files into pandas DataFrames.


**Reasoning**:
Import the pandas library and load the four data files into pandas DataFrames, paying attention to delimiters and headers, and then display the head of each DataFrame to verify the loading.



In [1]:
import pandas as pd

# Load NegativeEdges.txt
df_negative_edges = pd.read_csv("NegativeEdges.txt", sep='\t', header=None)
df_negative_edges.columns = ['protein1', 'protein2']

# Load PositiveEdges.txt
df_positive_edges = pd.read_csv("PositiveEdges.txt", sep='\t', header=None)
df_positive_edges.columns = ['protein1', 'protein2']

# Load proteinList.txt
df_protein_list = pd.read_csv("proteinList.txt", header=None)
df_protein_list.columns = ['protein']

# Load sequenceList.txt
df_sequence_list = pd.read_csv("sequenceList.txt", header=None)
df_sequence_list.columns = ['sequence']

print("Negative Edges:")
display(df_negative_edges.head())

print("\nPositive Edges:")
display(df_positive_edges.head())

print("\nProtein List:")
display(df_protein_list.head())

print("\nSequence List:")
display(df_sequence_list.head())

Negative Edges:


,protein1,protein2
0,4565,6450
1,6977,3142
2,6976,2889
3,2458,878
4,7151,1890



Positive Edges:


,protein1,protein2
0,0,5540
1,1,441
2,1,1117
3,1,1493
4,1,2189



Protein List:


,protein
0,0\t10968
1,1\t11542
2,2\t11540
3,3\t11544
4,4\t11548



Sequence List:


,sequence
0,MDGPVAEHAKQEPFHVVTPLLESWALSQVAGMPVFLKCENVQPSGS...
1,MLVCYSVLACEILWDLPCSIMGSPLGHFTWDKYLKETCSVPAPVHC...
2,MSRSRQPPLVTGISPNEGIPWTKVTIRGENLGTGPTDLIGLTICGH...
3,MDSLVTANTKFCFDLFQEIGKDDRHKNIFFSPLSLSAALGMVRLGA...
4,MAGGRGAPGRGRDEPPESYPQRQDHELQALEAIYGADFQDLRPDAC...


## Data exploration

### Subtask:
Explore the loaded DataFrames to understand their structure.


**Reasoning**:
Examine the shape, data types, missing values, duplicates, and other characteristics of the dataframes as per the instructions.



In [2]:
# Examine the shape of each DataFrame
print("Shape of df_negative_edges:", df_negative_edges.shape)
print("Shape of df_positive_edges:", df_positive_edges.shape)
print("Shape of df_protein_list:", df_protein_list.shape)
print("Shape of df_sequence_list:", df_sequence_list.shape)

# Check data types of each column
print("\nData types of df_negative_edges:\n", df_negative_edges.dtypes)
print("\nData types of df_positive_edges:\n", df_positive_edges.dtypes)
print("\nData types of df_protein_list:\n", df_protein_list.dtypes)
print("\nData types of df_sequence_list:\n", df_sequence_list.dtypes)

# Identify missing values
print("\nMissing values in df_negative_edges:\n", df_negative_edges.isnull().sum())
print("\nMissing values in df_positive_edges:\n", df_positive_edges.isnull().sum())
print("\nMissing values in df_protein_list:\n", df_protein_list.isnull().sum())
print("\nMissing values in df_sequence_list:\n", df_sequence_list.isnull().sum())

# Check for duplicates in df_protein_list
print("\nDuplicate entries in df_protein_list['protein']:", df_protein_list['protein'].duplicated().sum())

# Inspect sequence lengths in df_sequence_list
df_sequence_list['sequence_length'] = df_sequence_list['sequence'].str.len()
print("\nSequence length statistics:\n", df_sequence_list['sequence_length'].describe())

# Inspect protein IDs in edge lists and compare with protein list
# (Note: This requires more sophisticated analysis due to the format of df_protein_list)
# We'll check if protein IDs in edge lists are present in df_protein_list after splitting the 'protein' column
df_protein_list[['protein_id', 'extra']] = df_protein_list['protein'].str.split('\t', expand=True)
df_protein_list['protein_id'] = df_protein_list['protein_id'].astype(int)

negative_protein_ids = pd.concat([df_negative_edges['protein1'], df_negative_edges['protein2']]).unique()
positive_protein_ids = pd.concat([df_positive_edges['protein1'], df_positive_edges['protein2']]).unique()
all_edge_protein_ids = set(list(negative_protein_ids) + list(positive_protein_ids))

protein_ids_in_list = set(df_protein_list['protein_id'].unique())

missing_from_list = all_edge_protein_ids - protein_ids_in_list
print("\nProtein IDs from edges missing in protein list:", missing_from_list)

print("\nSummary:")
print("Note inconsistencies in protein IDs between edge lists and the protein list.")

Shape of df_negative_edges: (25203, 2)
Shape of df_positive_edges: (31761, 2)
Shape of df_protein_list: (7803, 1)
Shape of df_sequence_list: (7639, 1)

Data types of df_negative_edges:
 protein1    int64
protein2    int64
dtype: object

Data types of df_positive_edges:
 protein1    int64
protein2    int64
dtype: object

Data types of df_protein_list:
 protein    object
dtype: object

Data types of df_sequence_list:
 sequence    object
dtype: object

Missing values in df_negative_edges:
 protein1    0
protein2    0
dtype: int64

Missing values in df_positive_edges:
 protein1    0
protein2    0
dtype: int64

Missing values in df_protein_list:
 protein    0
dtype: int64

Missing values in df_sequence_list:
 sequence    0
dtype: int64

Duplicate entries in df_protein_list['protein']: 0

Sequence length statistics:
 count     7639.000000
mean       618.520225
std        634.030586
min         51.000000
25%        289.500000
50%        464.000000
75%        751.000000
max      26926.000000
N

## Data preparation

### Subtask:
Prepare the edge lists for graph construction.


**Reasoning**:
Combine the positive and negative edge dataframes, add the 'edge_type' column, and then check for consistency with the protein list and sequence list. Create the protein ID to sequence list index mapping.



In [3]:
# Concatenate positive and negative edges
df_all_edges = pd.concat([df_positive_edges, df_negative_edges], ignore_index=True)

# Add edge type column
df_all_edges['edge_type'] = ['positive'] * len(df_positive_edges) + ['negative'] * len(df_negative_edges)

# Check for consistency with protein list and create mapping if necessary
protein_id_mapping = dict(zip(df_protein_list['protein_id'], df_protein_list.index))
df_all_edges['protein1'] = df_all_edges['protein1'].map(protein_id_mapping).fillna(df_all_edges['protein1'])
df_all_edges['protein2'] = df_all_edges['protein2'].map(protein_id_mapping).fillna(df_all_edges['protein2'])


# Check for consistency with sequence list index
sequence_index_mapping = dict(zip(df_protein_list['protein_id'], df_protein_list.index))
# Verify consistency between df_all_edges and df_sequence_list indices

# Display the first few rows of df_all_edges
display(df_all_edges.head())

,protein1,protein2,edge_type
0,0,5540,positive
1,1,441,positive
2,1,1117,positive
3,1,1493,positive
4,1,2189,positive


## Data wrangling

### Subtask:
Ensure protein IDs in the edge lists match those in `proteinList.txt` and create a unique identifier for each protein.


**Reasoning**:
Verify the protein IDs in df_all_edges against df_protein_list and create unique identifiers. Handle inconsistencies by either ignoring or assigning new IDs to proteins missing in df_protein_list.  Then, update df_all_edges with the new unique identifiers.



In [4]:
# Use the index of df_protein_list as the unique identifier.
protein_id_to_unique_id = dict(zip(df_protein_list['protein_id'], df_protein_list.index))

# Map protein IDs in df_all_edges to unique IDs
df_all_edges['protein1_unique'] = df_all_edges['protein1'].map(protein_id_to_unique_id)
df_all_edges['protein2_unique'] = df_all_edges['protein2'].map(protein_id_to_unique_id)

# Check for missing unique IDs
missing_protein1 = df_all_edges['protein1_unique'].isnull().sum()
missing_protein2 = df_all_edges['protein2_unique'].isnull().sum()

print(f"Number of missing unique IDs for protein1: {missing_protein1}")
print(f"Number of missing unique IDs for protein2: {missing_protein2}")

if missing_protein1 > 0 or missing_protein2 > 0:
    print("Warning: Some protein IDs in df_all_edges were not found in df_protein_list.")
    # Handle missing unique IDs (e.g., assign a new unique ID or remove rows)
    # For now we will keep the original IDs for those missing proteins and fill na values with the original ID
    df_all_edges['protein1_unique'] = df_all_edges['protein1_unique'].fillna(df_all_edges['protein1'])
    df_all_edges['protein2_unique'] = df_all_edges['protein2_unique'].fillna(df_all_edges['protein2'])


# Verify the mapping
# Check if any unique identifiers are not in df_protein_list's index.
invalid_protein1_ids = set(df_all_edges['protein1_unique']) - set(df_protein_list.index)
invalid_protein2_ids = set(df_all_edges['protein2_unique']) - set(df_protein_list.index)

print(f"Invalid unique IDs for protein1: {invalid_protein1_ids}")
print(f"Invalid unique IDs for protein2: {invalid_protein2_ids}")


# Drop original protein ID columns and rename unique ID columns
df_all_edges = df_all_edges.drop(['protein1', 'protein2'], axis=1)
df_all_edges = df_all_edges.rename(columns={'protein1_unique': 'protein1', 'protein2_unique': 'protein2'})

display(df_all_edges.head())

Number of missing unique IDs for protein1: 0
Number of missing unique IDs for protein2: 0
Invalid unique IDs for protein1: set()
Invalid unique IDs for protein2: set()


,edge_type,protein1,protein2
0,positive,0,5540
1,positive,1,441
2,positive,1,1117
3,positive,1,1493
4,positive,1,2189


## Data preparation

### Subtask:
Create a protein sequence dictionary.


**Reasoning**:
Create the protein sequence dictionary based on the protein IDs and sequences from the corresponding dataframes.



In [5]:
# Create the protein sequence dictionary
protein_sequences = {}
for i in range(len(df_protein_list)):
    protein_id = df_protein_list.loc[i, 'protein_id']
    try:
      protein_sequences[protein_id] = df_sequence_list.loc[i, 'sequence']
    except IndexError:
      protein_sequences[protein_id] = "" # Handle cases where sequence is not available

# Print the length of the dictionary
print("Length of protein_sequences:", len(protein_sequences))

# Print the first 5 key-value pairs
print("First 5 key-value pairs:")
for i, (key, value) in enumerate(protein_sequences.items()):
    if i >= 5:
      break
    print(key, value[:20] + "...") #Print first 20 characters of sequence

KeyError: 7639

**Reasoning**:
The previous code failed due to an index error. The lengths of `df_protein_list` and `df_sequence_list` are different, so a direct index mapping won't work. Instead of using the index, we will use the `protein_id` as the key to map to the sequence. We need to handle the cases where there is no sequence associated with a given `protein_id`.



In [6]:
# Create the protein sequence dictionary using protein_id as key
protein_sequences = {}
for index, row in df_protein_list.iterrows():
    protein_id = row['protein_id']
    protein_sequences[protein_id] = ""  # Initialize with empty string

for index, row in df_sequence_list.iterrows():
    protein_id = df_protein_list.loc[index, 'protein_id'] # Map using index
    protein_sequences[protein_id] = row['sequence']

# Print the length of the dictionary
print("Length of protein_sequences:", len(protein_sequences))

# Print the first 5 key-value pairs
print("First 5 key-value pairs:")
for i, (key, value) in enumerate(protein_sequences.items()):
    if i >= 5:
        break
    print(key, value[:20] + "...")

Length of protein_sequences: 7803
First 5 key-value pairs:
0 MDGPVAEHAKQEPFHVVTPL...
1 MLVCYSVLACEILWDLPCSI...
2 MSRSRQPPLVTGISPNEGIP...
3 MDSLVTANTKFCFDLFQEIG...
4 MAGGRGAPGRGRDEPPESYP...


## Model training

### Subtask:
Build the protein-protein interaction network using NetworkX.


**Reasoning**:
Build the protein-protein interaction network using NetworkX based on the prepared dataframe `df_all_edges` and the protein sequences.



In [7]:
import networkx as nx

# Create a graph object (undirected since interactions are not specified as directed)
graph = nx.Graph()

# Add nodes with protein sequences as attributes
for protein_id in df_protein_list['protein_id'].unique():
  if protein_id in protein_sequences:
    graph.add_node(protein_id, sequence=protein_sequences[protein_id])

# Add edges with edge types as attributes
for index, row in df_all_edges.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    edge_type = row['edge_type']
    if protein1 in graph and protein2 in graph:
        graph.add_edge(protein1, protein2, edge_type=edge_type)


# Verify the graph
num_nodes = graph.number_of_nodes()
num_edges = graph.number_of_edges()
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

# Verify node attributes
# for node, data in graph.nodes(data=True):
#     if 'sequence' not in data:
#         print(f"Node {node} is missing sequence attribute.")

# Verify edge attributes
# for u, v, data in graph.edges(data=True):
#     if 'edge_type' not in data:
#         print(f"Edge ({u}, {v}) is missing edge_type attribute.")

Number of nodes: 7803
Number of edges: 56963


## Summary:

### Q&A
No questions were posed in the task description.

### Data Analysis Key Findings
* **Data Loading and Exploration**: Four data files (NegativeEdges.txt, PositiveEdges.txt, proteinList.txt, sequenceList.txt) were loaded into pandas DataFrames.  The protein list required processing to extract protein IDs.  No missing values were found in any dataframe. Sequence lengths in the sequence list varied considerably (51 to 26926, mean 618.52).
* **Edge List Preparation**: Positive and negative edge lists were combined, and an 'edge\_type' column was added. Protein IDs were mapped to ensure consistency with the protein list.  A mapping to sequence list indices was created, but consistency was not explicitly verified.
* **Unique Protein Identifiers**: Unique identifiers for proteins were created using the index of the protein list dataframe.  The original protein IDs in the edge list were replaced with these unique identifiers.
* **Protein Sequence Dictionary**: A dictionary mapping protein IDs to their sequences was created.  The dictionary was initialized with all protein IDs and then populated with sequences using index-based mapping.
* **Network Creation**: A protein-protein interaction network was built using NetworkX. The network has 7803 nodes (proteins) and 56963 edges (interactions).  Each node has a 'sequence' attribute, and each edge has an 'edge\_type' attribute.

### Insights or Next Steps
* The created protein-protein interaction network can now be used for further analysis, such as community detection, centrality analysis, or pathfinding.
* Investigate the potential biological implications of the positive and negative interactions in the network.
